In [10]:
import pandas as pd
import numpy as np
import os
import hdbscan

# Load the original data
dataset_path = '/Users/leo/Programming/PLR/Leo/data/cleaned_data_SYMPTOMS_9_13_23.csv'
original_data = pd.read_csv(dataset_path)

# Load the encoded data
encoded_data_path = '/Users/leo/Programming/PLR/Leo/main/final/grid_autoencoder/final_embedding/encoded_data_5_5.npy'
data_encoded = np.load(encoded_data_path)

# Apply HDBSCAN with the best parameters
clusterer = hdbscan.HDBSCAN(min_cluster_size=37, min_samples=6)
cluster_labels = clusterer.fit_predict(data_encoded)

# Add the cluster labels to the original data
original_data['cluster'] = cluster_labels

#save the data
original_data.to_csv('/Users/leo/Programming/PLR/Leo/main/final/full_data_clustered.csv', index=False)

In [9]:
# Create a new DataFrame with only 'Unnamed: 0' and 'cluster' columns
data_to_save = original_data[['Unnamed: 0', 'cluster']]

# Define the save directory and file name
save_dir = '/Users/leo/Programming/PLR/clusterings/leo'
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
save_path = os.path.join(save_dir, 'clustered_data.csv')

# Save the DataFrame to CSV
data_to_save.to_csv(save_path, index=False)
print(f"Data saved to {save_path}")

Data saved to /Users/leo/Programming/PLR/clusterings/leo/clustered_data.csv
